In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 가상 데이터 생성 (예: 100개 샘플, 10개 변수)
np.random.seed(0)
X = pd.DataFrame(np.random.randn(100, 10), columns=[f'x{i}' for i in range(1, 11)])
beta = np.random.randn(10)  # 진짜 계수
y = X @ beta + np.random.randn(100) * 0.5  # 선형 + 노이즈

# 상수항 추가
X = sm.add_constant(X)

def backward_elimination(X, y, verbose=True):
    variables = list(X.columns)
    best_aic = np.inf
    best_model = None
    
    while True:
        aic_with_vars = []
        
        # 각 변수 제거해가며 AIC 계산 (상수항(const) 제외)
        for var in variables:
            if var == 'const':
                continue
            vars_subset = [v for v in variables if v != var]
            X_subset = X[vars_subset]
            model = sm.OLS(y, X_subset).fit()
            aic_with_vars.append((model.aic, var, model))
        
        # 가장 AIC가 낮은 경우 선택
        aic_with_vars.sort(key=lambda x: x[0])
        best_new_aic, worst_var, candidate_model = aic_with_vars[0]
        
        if verbose:
            print(f"현재 변수: {variables}")
            print(f"제거 후보별 최소 AIC: {best_new_aic:.4f} (변수 제거: {worst_var})")
            print(f"이전 AIC: {best_aic if best_aic != np.inf else '무한대'}")
            print("---")
        
        # AIC 개선되면 변수 제거
        if best_new_aic < best_aic:
            variables.remove(worst_var)
            best_aic = best_new_aic
            best_model = candidate_model
        else:
            # 개선 안되면 종료
            break

    if verbose:
        print("최종 변수:", variables)
        print("최종 모델 요약:")
        print(best_model.summary())
    
    return best_model

# 실행
final_model = backward_elimination(X, y)


c:\users\vvv30\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


현재 변수: ['const', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10']
제거 후보별 최소 AIC: 150.5294 (변수 제거: x9)
이전 AIC: 무한대
---
현재 변수: ['const', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x10']
제거 후보별 최소 AIC: 153.9032 (변수 제거: x4)
이전 AIC: 150.5293796936627
---
최종 변수: ['const', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x10']
최종 모델 요약:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     253.9
Date:                Tue, 03 Jun 2025   Prob (F-statistic):           5.93e-60
Time:                        15:42:55   Log-Likelihood:                -65.265
No. Observations:                 100   AIC:                             150.5
Df Residuals:                      90   BIC:                             176.6
Df Model:         